<h1> Heatwaves on Yearly GDP</h1>

<p1>Ethan Jones, May 17, 2021</p1>

<p2>The following script stores heatwave functions designed for the 'Heatwave_on_GDP' file</p2>

In [1]:
import pandas as pd
import ipywidgets as widgets
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import pylab
from meteostat import Point, Daily
from meteostat import Stations
from datetime import datetime
from matplotlib.dates import DateFormatter, MonthLocator
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
from linearmodels import PanelOLS
import scipy.stats as stats
import seaborn as sns
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import statsmodels.formula.api as smf
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')

In [2]:
#join_data funciton: takes weather data and merges it with the country data

def join_data(df):
    url2 = 'https://gist.githubusercontent.com/ethanjones-git/b7a786e601f9f41d9edc8410fc851d18/raw/4da09c80106862241b7d8535a0bbc039f4807936/wb_country_info'
    
    econ=df.rename(columns={'country':'code'})
    country = pd.read_csv(url2).rename(columns={'country.1':'code'})
    
    econ['year'] = econ['year'].astype(str)
    country['year'] = country['year'].astype(str)

    
    econ['key'] =  econ['code'] + econ['year']
    country['key'] =  country['code'] + country['year']
    
    df =econ.merge(country, on = 'key', how = 'inner')
    df['year_x'] = df['year_x'].astype(int)
    df = df[["year_x",
        "temp_max",
        "total_days",
        'max_length',
        "country",
        "total_population",
        "urban_pop_ratio",
        "forest_area_ratio",
        "gdp_growth_rate",
        "gdp_growth_usd",
        "co2_emission_kt",
        "agri_land_ratio",
        "incomeLevel",
        "land_area_sq_km"]]
    df = df.rename(columns={'year_x':'year'})
    
    na_perc = (pd.DataFrame(df.isnull().sum()/len(df)))
    na_perc.plot(figsize=(15,5), kind='bar',legend= False, title= "Percentage of Data Missing Per Category")
    
    # Initialise the subplot function using number of rows and columns
    figure, axis = plt.subplots(3, 2,figsize=(10,10))
    figure.tight_layout()
  
    # For heatwaves Function
    axis[0,0].hist(df.temp_max)
    axis[0,0].set_title("temp_max")
  
    # Avg duration
    axis[0,1].hist(df.total_days)
    axis[0,1].set_title("total_days")

    # Max heatwave days
    axis[1,0].hist(df.max_length)
    axis[1,0].set_title("max_length")
  
    # Total Population
    axis[1,1].hist(df.total_population)
    axis[1,1].set_title("total_population")
  
    # GDP Growth USD
    axis[2,1].hist(df.gdp_growth_usd)
    axis[2,1].set_title("gdp_growth_usd")

    # GDP Growth Rate
    axis[2,0].hist(df.gdp_growth_rate)
    axis[2,0].set_title("gdp_growth_rate")
  
    # Combine all the operations and display
    plt.show()
    df = pd.DataFrame(df)
  
   
    return df


#take raw y_test and predict, turns them into a graph

def predict_plot(y_test, predicter):
    ytest= pd.DataFrame(y_test)
    predicter= pd.DataFrame(predicter)
    ytest.reset_index(level=0, inplace= True)
    df =ytest.merge(predicter, left_index = True, right_index = True)
    df.columns = ['year','test','predict']
    df = df.sort_values(by=['year'],ascending=True)

    plt.plot(df.year,df.test, label = "test")
    plt.plot(df.year, df.predict, label = "prediction")
    plt.legend()
    plt.show()
    
def vis(country): 
    url = 'https://gist.githubusercontent.com/ethanjones-git/640b9912b985e49a6ac6bbdad2abacba/raw/d4b1a79e3dc933751642c8adc0f27a158348051e/dim_hw_country_static_info'
    df_country_info = pd.read_csv(url,index_col = 'country')
    code =  df_country_info.loc[country]['alpha3_code']
    
    url = 'https://gist.githubusercontent.com/ethanjones-git/640b9912b985e49a6ac6bbdad2abacba/raw/d4b1a79e3dc933751642c8adc0f27a158348051e/dim_hw_country_static_info'
    df_country_info = pd.read_csv(url,index_col = 'alpha3_code')
    
    num_of_stations =  df_country_info.loc[code]['num_of_weather_station']
    num_of_stations = np.where(num_of_stations>100,100,num_of_stations).tolist()
    start_date = datetime(1960, 1, 1)
    end_date = datetime(2020, 11, 1)
    
    
    stations = list(Stations().region(df_country_info.loc[code]['alpha2_code']).fetch(num_of_stations).index)
    
    df_temp_data = Daily(stations, start_date, end_date).fetch()

    df1=df_temp_data[['tavg','tmin','tmax']]
    
    #graph
    p1=df_temp_data.groupby('time')[['tmax']].mean().plot(figsize=(15,5),title = "Complete Daily Temperature Max for " + country)
    

    df1 = df1.groupby('time')['tmax'].mean().reset_index()

    #creates a month day index
    df1['mday'] = df1['time'].dt.month.astype(str)+ '/' + df1['time'].dt.day.astype(str)
    df1['year'] = df1['time'].dt.year

    #reference time 
    df2= df1.loc[(df1['time'].dt.year <= 1990) & (df1["time"].dt.year >= 1960)]

    #90th quantile for every day in the 30 year period
    df2 = df2.groupby('mday')['tmax'].quantile(.9).reset_index()

    #rolling 15-day average
    df2['rollin'] = df2['tmax'].rolling(15).mean()

    #subset to important months
    df1 = df1.merge(df2,on ='mday',how = 'left')

    df1 = df1[['time','tmax_x','rollin']]
    df1.columns = ['date','temp_max','per']
    
    #graphs
    years = [1960,1980,2000]

    x2 = df1.loc[(df1['date'].dt.year <= 2020) & (df1['date'].dt.year >= 2020)].groupby('date')[['temp_max','per']].max()
    x2['date'] = x2.index
    x2['MM-DD'] = x2['date'].dt.strftime('%m-%d')


    for x in years:
        x1 = df1.loc[(df1['date'].dt.year <= x) & (df1['date'].dt.year >= x)].groupby('date')[['temp_max','per']].max()
        x1['date'] = x1.index
        x1['MM-DD'] = x1['date'].dt.strftime('%m-%d')
        x2= x1.merge(x2, on = 'MM-DD', how = 'left')

    x2=x2.iloc[:,[0,2,1,4,7,10]]
    x2.columns = ['max2000','date','threshold','max1980','max1960','max2020']
    
    fig, axs = plt.subplots(2,2,figsize=(15,15))
    months = MonthLocator()
    monthsFmt = DateFormatter("%b")
    #fig.legend(handles, labels, loc='upper center')

    fig.suptitle(country+" "+'Heatwave Threshold (Blue) vs Daily Temperature Max, Four Samples')
    
    axs[0,0].plot(x2['date'], x2['max1960'], linewidth=0.5, color = 'black',alpha= 0.5)
    axs[0,0].plot(x2['date'], x2['threshold'], linewidth=2, color = 'blue', alpha = 0.8)
    axs[0,0].set_title("1960")
    axs[0,0].xaxis.set_major_locator(months)
    axs[0,0].xaxis.set_major_formatter(monthsFmt)
    #axs[0,0].legend(('yearly max temp', 'heatwave threshold'), loc='upper right', shadow=True)

    axs[0,1].plot(x2['date'], x2['max1980'], linewidth=0.5, color = 'black',alpha= 0.5)
    axs[0,1].plot(x2['date'], x2['threshold'], linewidth=2, color = 'blue', alpha = 0.8)
    axs[0,1].set_title("1980")
    axs[0,1].xaxis.set_major_locator(months)
    axs[0,1].xaxis.set_major_formatter(monthsFmt)
    
    axs[1,0].plot(x2['date'], x2['max2000'], linewidth=0.5, label = '1960', color = 'black',alpha= 0.5)
    axs[1,0].plot(x2['date'], x2['threshold'], linewidth=2, color = 'blue', alpha = 0.8)
    axs[1,0].set_title("2000")
    axs[1,0].xaxis.set_major_locator(months)
    axs[1,0].xaxis.set_major_formatter(monthsFmt)

    axs[1,1].plot(x2['date'], x2['max2020'], linewidth=0.5, label = '1980', color = 'black',alpha= 0.5)
    axs[1,1].plot(x2['date'], x2['threshold'], linewidth=2, color = 'blue', alpha = 0.8)
    axs[1,1].set_title("2020")
    axs[1,1].xaxis.set_major_locator(months)
    axs[1,1].xaxis.set_major_formatter(monthsFmt)
    fig.legend(('yearly max temp', 'heatwave threshold'), loc='lower right', shadow=True)

def country_by_name(country):
    
    url = 'https://gist.githubusercontent.com/ethanjones-git/640b9912b985e49a6ac6bbdad2abacba/raw/d4b1a79e3dc933751642c8adc0f27a158348051e/dim_hw_country_static_info'
    df_country_info = pd.read_csv(url,index_col = 'alpha3_code')
    
    num_of_stations =  df_country_info.loc[country]['num_of_weather_station']
    num_of_stations = np.where(num_of_stations>100,100,num_of_stations).tolist()
    start_date = datetime(1960, 1, 1)
    end_date = datetime(2020, 11, 1)
    
    
    stations = list(Stations().region(df_country_info.loc[country]['alpha2_code']).fetch(num_of_stations).index)
    
    df_temp_data = Daily(stations, start_date, end_date).fetch()

    df =df_temp_data[['tavg','tmin','tmax']]

    #averages all stations by the same date
    #def raw_heatwave_conditions(df1):
    df1 = df.groupby('time')['tmax'].mean().reset_index()

    #creates a month day index
    df1['mday'] = df1['time'].dt.month.astype(str)+ '/' + df1['time'].dt.day.astype(str)
    df1['year'] = df1['time'].dt.year

    #reference time 
    df2= df1.loc[(df1['time'].dt.year <= 1990) & (df1["time"].dt.year >= 1960)]

    #90th quantile for every day in the 30 year period
    df2 = df2.groupby('mday')['tmax'].quantile(.9).reset_index()

    #rolling 15-day average
    df2['rollin'] = df2['tmax'].rolling(15).mean()

    #subset to important months
    df1 = df1.merge(df2,on ='mday',how = 'left')

    df1 = df1[['time','tmax_x','rollin']]
    df1.columns = ['date','temp_max','per']
    
    #return df1


    #def hw_subset(df1):

    #hot months
    cond1 = df1['date'].dt.month > 4
    cond2 = df1['date'].dt.month < 10

    #above 90 percentile
    cond3 = df1["per"] < df1["temp_max"]

    #create dataframe
    df2 = df1[cond1 & cond2 & cond3] 


    cond4 = (df2['date'] - df2['date'].shift(-1)) < '-1 days'
    df2['count'] = np.where(cond4, 0,1)
    df2['sum_count'] = df2.groupby(df2['count'].shift().eq(0).cumsum()).cumcount()


    #cond6= np.where(df2['sum_count'].shift(-1) == 0,df2['count'].shift(2) == 0,1)
    df2['sum_count'] = np.where(df2['sum_count']==1,0,df2['sum_count'])

    df2['sum_count'] = np.where(df2['sum_count'] == 0, df2['sum_count'].shift(1),0)
    df2['year'] = df2['date'].dt.year
    
    #freq sum of all heatwave days
    df2_avg= df2.groupby('year')['temp_max'].mean().reset_index()

    #intensity, avg across all days
    df2_freq = df2.groupby('year')['sum_count'].sum().reset_index()

    #duration is the longest event
    df2_lng = df2.groupby('year')['sum_count'].max().reset_index()


    df =df2_avg.merge(df2_freq, on = 'year', how = 'left')
    df =df.merge(df2_lng, on = 'year', how = 'left')
    df.columns = ['year','temp_max','total_days','max_length']
    df['country'] = country
    return df
    
def forcast(year_choose,temp_max,total_days,max_length,wld_gdp,cont,df):
    
    #merge existing features with country demographics
    url4 = "https://gist.githubusercontent.com/ethanjones-git/b7a786e601f9f41d9edc8410fc851d18/raw/4da09c80106862241b7d8535a0bbc039f4807936/wb_country_info"
    wb = pd.read_csv(url4).rename(columns = {'country':'code',
                                             'country.1':'country'})[['total_population',
                                                                  'country',
                                                                  'land_area_sq_km']]

    df = df.merge(wb, how ='left', on =['country'])

    #Categorical ML
    dummies = pd.get_dummies(df.country)
    merged = pd.concat([df,dummies],1)
    final = merged.drop(['country','USA'],1)

    #create training and test sets
    df_data = final.drop(['gdp','subregion'],1)
    df_tar = final['gdp']
    x_train,x_test,y_train,y_test = train_test_split(df_data,df_tar,test_size = 0.2, random_state=0)
    
    #model
    model= Pipeline(steps=[('scalar1', StandardScaler()),
                ('func',
                 ElasticNet(alpha=0.01, max_iter=5000, selection='random'))])
    model.fit(df_data,df_tar)
    m= model.predict(x_test)
    mse =mean_squared_error(y_test,m)
    
    
    #predicition format
    pred = x_test.head(1)
    pred.loc[0, ['year','temp_max','total_days','max_length','wld_gdp',cont]] = [year_choose,temp_max,total_days,max_length,wld_gdp,1]
    pred = pd.DataFrame(pred.loc[0,:].fillna(0)).T


    #take past contry data
    dff = df.loc[df['country']==cont]


    #join w/ predicition format
    kk = pd.concat([pred,dff]).drop(['country','subregion'],1)
    ll= kk.fillna(0)
    ll.loc[:,cont]= 1


    #prepare for ML use
    ll.sort_values(by = ['year'])
    pred = ll.drop('gdp',1).sort_values(by='year')
    m= model.predict(pred)

    #create estimate(s) under ['gdp']
    pred['gdp'] = m


    #predicted values
    val = pred.loc[pred['year']==year_choose]
    pp = str(val['gdp'])

    #create plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pred['year'], y=pred['gdp'],
                        mode='lines+markers',
                        name='Predicted GDP Change Rate',
                        line_color='teal'))
    fig.add_trace(go.Scatter(x=dff['year'], y=dff['gdp'],
                    mode='lines+markers',
                    name='Real GDP Change Rate'))
    fig.add_trace(go.Scatter(x=val['year'], y=val['gdp'],
                    mode='markers', name=('Forcast for'+' '+ cont)))
    fig.update_layout(title={'text': "Prediction By Country",
                                'y':0.85,
                                'x':0.4,
                                'xanchor': 'center',
                                'yanchor': 'top'},
                      xaxis_title="Year",
                      yaxis_title="GDP Change Rate")

    fig.show()
    
def pannel_obs(subregion):

    
    #widget
    df = pd.read_csv('https://gist.githubusercontent.com/ethanjones-git/bf881e15089e88a1962e82578b546974/raw/0ff947fc3d8b0bbf08601b48aa4d389fda70e674/subregion')
    options = set(df['subregion'])
    
    #subset by subregion
    mm = df[(df['subregion']==subregion)|(df['country']=='WLD')]

    #country plot
    fig = px.choropleth(mm, locations="country",
                        hover_name="country",
                        title = ('Selected Region'))
    fig.update_layout(showlegend=False)
    fig.update_layout(title={'text': "Selected Region",
                                    'y':0.85,
                                    'x':0.29,
                                    'xanchor': 'right',
                                    'yanchor': 'top'})
    fig.show()
    #mm.groupby['country']
    dff = mm.set_index(['country','year'])
    dff['const'] = 1
    features = ['temp_max','total_days','max_length']
    a= []
    k =[]
    for f in features:
        m = f
        model = PanelOLS(dff['gdp'],dff[[m,'wld_gdp','const']]).fit(cov_type='clustered', entity_effects=True, time_effects = True)
        mm = pd.DataFrame(model.summary.tables[1]).drop(0,0).rename(columns={0:"Variables",
                                                                                1:"Coefficent",
                                                                                2:"Standard_Error",
                                                                                3:"T_Statistic",
                                                                                4:"P_Value",
                                                                                5:"Lower_CI",
                                                                                6:"Upper_CI",
                                                                                'feature':'Feature'})
        t = pd.DataFrame(model.resids)
        mm['feature'] = f
        t['feature']= f
        k.append(mm)
        a.append(t)
    mm =pd.concat(k)
    t.reset_index(inplace=True)


    #results 
    m1=mm[:3]
    m2=mm[3:6]
    m3=mm[6:9]
    fig =  ff.create_table(m1)
    fig.update_layout(title={'text': "Maximum Heat Wave Temperature",
                                    'y':0.8,
                                    'x':0.31,
                                    'xanchor': 'right',
                                    'yanchor': 'top'})
    fig.update_layout({'margin':{'t':50}})
    fig.show()
    fig =  ff.create_table(m2)
    fig.update_layout(title={'text': "Total Heat Wave Days",
                                    'y':0.8,
                                    'x':0.25,
                                    'xanchor': 'right',
                                    'yanchor': 'top'})
    fig.update_layout({'margin':{'t':50}})

    fig.show()
    fig =  ff.create_table(m3)
    fig.update_layout(title={'text': "Maximum Heatwave Length",
                                    'y':0.8,
                                    'x':0.24,
                                    'xanchor': 'right',
                                    'yanchor': 'top'})
    fig.update_layout({'margin':{'t':50}})
    fig.show()

    #line plot
    k = dff.groupby('year').mean()
    k['temp_max'] = k.temp_max.rank(pct = True)
    k['total_days']= k.total_days.rank(pct = True)
    k['max_length'] = k.max_length.rank(pct = True)
    k['year']=k.index
    
    fig = px.line(t, x="year", y="residual", color='country')
    fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Residuals: Total Heat Wave Days ")
    fig.update_layout(title={'text': "Variation Isolated by Treatment: Country Specific Contributions to Feature Coefficent",
                                    'y':0.92,
                                    'x':0.45,
                                    'xanchor': 'center',
                                    'yanchor': 'top'})
    fig.show()
    dff.reset_index(inplace=True)
    fig = px.bar(dff, x='year', y='total_days',
                           color='country')
    fig.update_layout(title={'text': "Total Heat Wave Days Per Year",
                                    'y':0.92,
                                    'x':0.37,
                                    'xanchor': 'right',
                                    'yanchor': 'top'})

    fig.show()

def graph_missing_years(df):
    m = pd.DataFrame(df.groupby('country')['year'].nunique())
    m['per_missin'] = m['year']/59
    m['per_missin'] = 1 - m['per_missin']
    m.sort_values('per_missin')

    fig = px.histogram(m, y=m.index, x=m["per_missin"],
                       orientation='h',
                       hover_data=m.columns)
    fig.update_layout(uniformtext_minsize=2,
                      barmode='stack', yaxis={'categoryorder':'total descending'},
                       title={
                                'text': "Missing Data By Country",
                                'y':0.95,
                                'x':0.5,
                                'xanchor': 'center',
                                'yanchor': 'top'},
                      xaxis_title="Percentage of Data Missing",
                      yaxis_title="Country Codes")
    fig.add_vrect(x0=0.2, x1=0.21, line_width=0, fillcolor="red", opacity=0.2)
    return fig.show()